### Задание 1
Напишите декоратор, оптимизирующий работу декорируемой функции. Декоратор должен сохранять результат работы функции на ближайшие три запуска и вместо выполнения функции возвращать сохранённый результат. 
После трёх запусков функция должна вызываться вновь, а результат работы функции — вновь кешироваться.

### Анализ
Будем хранить результат вычисления функции и возвращать его вместо вызова самой функции. Как некий ключ будем использовать параметры функции чтобы при изменении параметров функция вновь пересчитала результирующее значение

In [31]:
def save_result(func):
    cache = {}
    call_counts = {}
    max_calls = 3

    def wrapper(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        if key not in call_counts:
            call_counts[key] = 0

        if call_counts[key] < max_calls and key in cache:
            call_counts[key] += 1
            print(f"Кеширующий результат с параметрами: {', '.join(map(str, args))}")

            return cache[key]
        else:
            result = func(*args, **kwargs)
            cache[key] = result

            return result

    return wrapper

@save_result
def pow2(x):
    print(f"Выполнение функции с параметрами: {x}")
    return x * x


print(pow2(2))
print(pow2(2))
print(pow2(3))
print(pow2(3))
print(pow2(2))
print(pow2(2))
print(pow2(2))


Выполнение функции с параметрами: 2
4
Кеширующий результат с параметрами: 2
4
Выполнение функции с параметрами: 3
9
Кеширующий результат с параметрами: 3
9
Кеширующий результат с параметрами: 2
4
Кеширующий результат с параметрами: 2
4
Выполнение функции с параметрами: 2
4


### Вывод
Реализовали кеширующую функцию. По логам видно, что 1 вызов с параметром 2 был вычислен. Затем при повторном вызове результат был взят из кеша. При вызове функции с параметром 3 результат был корректно вычислен и при повторном вызове с 3 взят из кеша. После этого при вызове с параметром 2 программа продолжила корректно работать и взяла результат из кеша. На 4 раз при вызове с параметром 2 результат был пересчитан. 

### Задание 2
Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов.

### Анализ
Запустим локальный сервер для раздачи статических файлов с помощью команды. Нет необходимости dosить сайты

`python3 -m http.server 8000 --bind 127.0.0.1`

Сгенерируем картинки из любого видео с помощью команды

`ffmpeg -i someVideo.mp4 %04d.bmp`

По предварительной оценке последовательный метод будет самым медленным. Особой разницы между мультипотоками и мультипроцессами не ожидается так как не требуется больших вычислений и время создания процесса мало по сравнению с загрузкой файла. В таком случае мультипроцессорны вариант выглядит более предпочтительным.

In [ ]:
!ffmpeg -i test.webm ./img/%04d.bmp
!python3 -m http.server 8000 --bind 127.0.0.1

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [51]:
import requests
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
import time

def download_file(url):
    requests.get(url)

def sequential_download(urls):
    for url in urls:
        download_file(url)

def threaded_download(urls, num_threads):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(download_file, urls)

def process_download(urls, num_processes):
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(download_file, urls)

urls =  [f"http://127.0.0.1:8000/img/{i:04}.bmp" for i in range(1, 500)]

start_time = time.time()
sequential_download(urls)
print(f"Последовательная загрузка заняла {time.time() - start_time} секунд")

start_time = time.time()
threaded_download(urls, 2)
print(f"Мультипоточная загрузка заняла {time.time() - start_time} секунд")

start_time = time.time()
process_download(urls, 8)
print(f"Мультипроцессорная загрузка заняла {time.time() - start_time} секунд")

Последовательная загрузка заняла 8.706597805023193 секунд
Мультипоточная загрузка заняла 4.488717317581177 секунд
Мультипроцессорная загрузка заняла 1.7495882511138916 секунд


### Вывод
На Ryzen 9 5950X были получены следующие результаты:
- Оптимальное количество потоков для мультипоточной заргузки: 2. При дальнейшем увеличении числа потоков время увеличивается. 
- Оптимальное число процессов: 8. Далее время перестает уменьшаться.

Результаты не соответствуют начальным предположениям. 